# Import

In [47]:
import torch
import numpy as np
from itertools import pairwise

# Mean average precision

In [50]:
eps = 1e-4

In [56]:
all_boxes = [
    {
        'pred': [
            [0, 0, 0.9, 0.55, 0.2, 0.3, 0.2],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2]
        ],
        'true': [
            [0, 0, 0.9, 0.55, 0.2, 0.3, 0.2],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2]
        ],
        'iou_threshold': 0.5,
        'format': 'midpoint',
        'num_classes': 1,
        'map': 1
    },
    {
        'pred': [
            [1, 0, 0.9, 0.55, 0.2, 0.3, 0.2],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2]
        ],
        'true': [
            [1, 0, 0.9, 0.55, 0.2, 0.3, 0.2],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2]
        ],
        'iou_threshold': 0.5,
        'format': 'midpoint',
        'num_classes': 1,
        'map': 1
    },
    {
        'pred': [
            [0, 1, 0.9, 0.55, 0.2, 0.3, 0.2],
            [0, 1, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 1, 0.7, 0.8, 0.7, 0.2, 0.2]
        ],
        'true': [
            [0, 0, 0.9, 0.55, 0.2, 0.3, 0.2],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2]
        ],
        'iou_threshold': 0.5,
        'format': 'midpoint',
        'num_classes': 2,
        'map': 0
    },
    {
        'pred': [
            [0, 0, 0.9, 0.15, 0.25, 0.1, 0.1],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2]
        ],
        'true': [
            [0, 0, 0.9, 0.55, 0.2, 0.3, 0.2],
            [0, 0, 0.8, 0.35, 0.6, 0.3, 0.2],
            [0, 0, 0.7, 0.8, 0.7, 0.2, 0.2]
        ],
        'iou_threshold': 0.5,
        'format': 'midpoint',
        'num_classes': 1,
        'map': 5 / 18
    },
]

In [57]:
from yolo.src.metrics import iou

In [91]:
def mean_average_precision(pred, true, iou_threshold, format: str = 'pascal_voc', num_classes: int = 20):
    auc_values = []
    for class_num in range(num_classes):
        boxes = [box for box in pred if box[1] == class_num]
        boxes = sorted(boxes, key=lambda x: x[2], reverse=True) # sort by score
        TP = FP = 0
        precision = [1]
        recall = [0]
        for pred_box in boxes:
            image_num = pred_box[0]
            gt_boxes = [box for box in true if box[0] == image_num and box[1] == class_num]
            max_iou, corresponding_box_index = 0, -1
            for i, gt_box in enumerate(gt_boxes):
                current_iou = iou(torch.tensor(pred_box[3:]).unsqueeze(0), torch.tensor(gt_box[3:]).unsqueeze(0), format)[0].item()
                if current_iou > max_iou:
                    max_iou = current_iou
                    corresponding_box_index = i
            if corresponding_box_index == -1 or max_iou < iou_threshold:
                FP += 1
            else:
                TP += 1
                gt_boxes.pop(corresponding_box_index)
            precision.append(TP / (TP + FP))
            recall.append(TP / len(boxes))
        auc_values.append(torch.trapz(torch.tensor(precision), torch.tensor(recall)).item())
    return np.mean(auc_values)

In [92]:
for i, box in enumerate(all_boxes):
    print(f"\nTest {i + 1}")
    mean_avg_precision = mean_average_precision(box['pred'], box['true'], box['iou_threshold'], box['format'], box['num_classes'])
    print(box['map'], mean_avg_precision)


Test 1
1 1.0

Test 2
1 1.0

Test 3
0 0.0

Test 4
0.2777777777777778 0.2777777910232544
